In [ ]:
# Create synaptograms from an input json file 

In [1]:
import numpy as np 
from at_synapse_detection import synaptogram
from at_synapse_detection.render_module import RenderModule
import renderapi
import matplotlib.pyplot as plt
from at_synapse_detection import dataAccess as da
from at_synapse_detection import SynapseDetection as syn 
import os
import json 
from at_synapse_detection import evaluate_synapse_detection as esd
from at_synapse_detection.AnnotationJsonSchema import AnnotationFile, NumpyArray
import importlib
from at_synapse_detection import processDetections as pd


In [ ]:
data = json.load(open('../data/M247514_Rorb_1/Site3Align2/json_annotations/m247514_Site3Annotation_MN_global_v2.json'))

In [ ]:
listOfSynapses = data['area_lists']

In [ ]:
synapse = listOfSynapses[0]

In [222]:
synapse['oid']

'2840'

In [ ]:
win_xy = 4
win_z = 1

In [ ]:
bbox = synaptogram.getAnnotationBoundingBox2(synapse)
bbox = synaptogram.transformSynapseCoordinates(bbox)
expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

In [ ]:
synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

In [ ]:
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/';
stackList = ['results', 'PSD95', 'synapsin', 'VGlut1', 'GluN1', 'GABA', 'Gephyrin']

In [ ]:
showProb = True
#filename = 'foo.png'
filename = os.path.join(filepath, '{}.png'.format(synapse['oid']))

In [ ]:
img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);

In [ ]:
%matplotlib notebook

plt.imshow(img, cmap='gray')
plt.colorbar()

In [ ]:
textXOffset = 1
textYOffset = 1

In [ ]:
synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, 0, 5)
#plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, 0, 5)

In [140]:
metadataFN = '../data/M247514_Rorb_1/Site3Align2/site3_metadata.json'
metadata = syn.loadMetadata(metadataFN)

datalocation = metadata['datalocation']
queryFN = metadata['querylocation']

# List of Queries
listOfQueries = syn.loadQueriesJSON(queryFN)


In [ ]:
## RUNME 

In [7]:
args = {
    "EM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/json_annotations/m247514_Site3Annotation_MN_global_v2.json",
    "LM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/results/resultVol9.json",
    "EM_metadata_csv":"../data/M247514_Rorb_1/Site3Align2/MNSite3Synaptograms_v2.csv",
    "LM_metadata_file":"../data/M247514_Rorb_1/Site3Align2/site3_metadata.json",
    "EM_inclass_column":"glutsynapse",
    "EM_not_synapse_column":"ConsensusNotSynapse",
    "output_json":"../data/M247514_Rorb_1/Site3Align2/results/Anish_evaluation_output.json"
    }

In [8]:
# COMBINE QUERIES

In [14]:
print(listOfQueries[5])
args['LM_annotation_json'] = "../data/M247514_Rorb_1/Site3Align2/results_07/resultVol5.json"
queryresult1 = pd.evalsyndetections(args)
missedAnnoIds_q2 = pd.getMissedAnnoIds(queryresult1['missed_annotations'])
len(missedAnnoIds_q2)

{'postIF_z': [1], 'preIF_z': [3], 'postIF': ['PSD95'], 'preIF': ['synapsin']}
EM_per_LM [ 0.27055703  0.57824934  0.15119363]
LM_per_EM [ 0.32457496  0.66615147  0.00927357]
lm edge detections: 3920
em edge annotations 156
LM detections: 4313


210

In [15]:
print(listOfQueries[6])
args['LM_annotation_json'] = "../data/M247514_Rorb_1/Site3Align2/results_07/resultVol6.json"
queryresult2 = pd.evalsyndetections(args)
missedAnnoIds_q3 = pd.getMissedAnnoIds(queryresult2['missed_annotations'])
len(missedAnnoIds_q3)

{'postIF_z': [3], 'preIF_z': [1], 'postIF': ['PSD95'], 'preIF': ['synapsin']}
EM_per_LM [ 0.02409639  0.81024096  0.16566265]
LM_per_EM [ 0.37094281  0.62132921  0.00772798]
lm edge detections: 3482
em edge annotations 156
LM detections: 3815


240

In [192]:
def combineResultVolumes(listOfQueryNumbers, listOfThresholds, args): 

    resultVolList = [] 
    for n, queryNum in enumerate(listOfQueryNumbers): 
    
        fn = os.path.join(metadata['datalocation'], 'resultVol')
        fn = fn + str(queryNum) + '.npy'
        resultVol_n = np.load(fn)
        print(fn)

        resultVol_n = resultVol_n > listOfThresholds[n]
        resultVolList.append(resultVol_n)

    resultVol = resultVolList[0]
    combinedQNum = str(listOfQueryNumbers[0]) 
    for volItr in range(1, len(resultVolList)): 
        resultVol = resultVol + resultVolList[volItr]
        combinedQNum = combinedQNum + str(listOfQueryNumbers[volItr])
        
    combinedQNum = combinedQNum + str(0) + str(0) 
    
    pd.probMapToJSON(resultVol, metadata, combinedQNum)
    
    fn = "../data/M247514_Rorb_1/Site3Align2/results/resultVol" + combinedQNum + ".json"
    
    args['LM_annotation_json'] = fn
    queryresult = pd.evalsyndetections(args)
    missedAnnoIds = pd.getMissedAnnoIds(queryresult['missed_annotations'])
    len(missedAnnoIds)

    return queryresult
    

In [143]:
listOfQueries[22]

{'postIF': ['PSD95', 'GluN1'], 'postIF_z': [3, 1], 'preIF': [], 'preIF_z': []}

In [258]:
listOfQueryNumbers = [  3,   4,   6,  11,   15,  19,  20,  21]
listOfThresholds   = [0.8, 0.7, 0.7, 0.8,  0.7, 0.8, 0.9, 0.9]

In [259]:
queryresult = combineResultVolumes(listOfQueryNumbers, listOfThresholds, args)

/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol3.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol4.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol6.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol11.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol15.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol19.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol20.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol21.npy
EM_per_LM [ 0.18811881  0.62871287  0.18316832]
LM_per_EM [ 0.20710974  0.78979907  0.00309119]
lm edge detections: 4554
em edge annotations 156
LM detections: 4971


In [245]:
listOfQueryNumbersPSD = [0]
listOfThresholdsPSD   = [0.9]
queryresultPSD = combineResultVolumes(listOfQueryNumbersPSD, listOfThresholdsPSD, args)

/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol0.npy
EM_per_LM [ 0.64797136  0.26252983  0.08949881]
LM_per_EM [ 0.07418856  0.91499227  0.01081917]
lm edge detections: 10116
em edge annotations 156
LM detections: 10975


In [246]:
missedAnnoIdsPSD = pd.getMissedAnnoIds(queryresultPSD['missed_annotations'])


In [247]:
missedAnnoIds = pd.getMissedAnnoIds(queryresult['missed_annotations'])

In [248]:
len(missedAnnoIds)

133

In [249]:
itr = 0 

old_missed_annotations =  queryresult['missed_annotations']
missed_annotations = []
for n, nID in enumerate(missedAnnoIds): 
    if nID in missedAnnoIdsPSD: 
        foo = 1 
    else: 
        missed_annotations.append(old_missed_annotations[n])
        itr = 1 + itr
        
print(itr)
        

88


In [217]:
len(missed_annotations)

87

In [172]:
em_edge = queryresult['EM_edge']
np.sum(em_edge)

156

In [80]:
win_xy = 4
win_z = 1
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/';
stackList = ['results', 'PSD95', 'synapsin', 'VGlut1', 'GluN1', 'GABA', 'Gephyrin']
showProb = True
textXOffset = 0
textYOffset = 5



In [ ]:
# for counter, synapse in enumerate(good_annotations): 
    
#     if (LM_per_EM[counter] > 0 and EM_edge[counter]==False): 
#         bbox = synaptogram.getAnnotationBoundingBox2(synapse)
#         bbox = synaptogram.transformSynapseCoordinates(bbox)
#         expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

#         synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
#         synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

#         filename = os.path.join(filepath, 'synaptograms/good', '{}.png'.format(synapse['oid']))

#         img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);
#         synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, textXOffset, textYOffset)
#         print(filename)

In [250]:
for counter, synapse in enumerate(missed_annotations): 
    
    #if synapse['oid'] in missed_anno: 
    bbox = synaptogram.getAnnotationBoundingBox2(synapse)
    bbox = synaptogram.transformSynapseCoordinates(bbox)
    expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

    synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
    synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

    filename = os.path.join(filepath, 'synaptograms/missed', '{}.png'.format(synapse['oid']))

    img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);
    synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, textXOffset, textYOffset)
    print(filename)

/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/581.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/623.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/631.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/635.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/691.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/781.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/819.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/821.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/827.png
/Users/ani

/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2434.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2442.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2474.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2478.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2516.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2532.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2534.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2582.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/missed/2586.png
/

In [ ]:
#false postives 

In [251]:
false_positives = queryresult['false_positives']

In [252]:
false_positives[0]

{'areas': [{'global_path': array([[ 8958.33333333,  5091.66666667],
          [ 8958.33333333,  5175.        ],
          [ 9008.33333333,  5175.        ],
          [ 9008.33333333,  5091.66666667],
          [ 8958.33333333,  5091.66666667]]), 'z': 39.0},
  {'global_path': array([[ 8958.33333333,  5091.66666667],
          [ 8958.33333333,  5175.        ],
          [ 9008.33333333,  5175.        ],
          [ 9008.33333333,  5091.66666667],
          [ 8958.33333333,  5091.66666667]]), 'z': 40.0}],
 'id': 1823,
 'oid': '1823'}

In [253]:
for counter, synapse in enumerate(false_positives): 

    bbox = synaptogram.getAnnotationBoundingBox2(synapse)
    bbox = synaptogram.transformSynapseCoordinates(bbox)
    expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

    synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
    synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

    filename = os.path.join(filepath, 'synaptograms/false_positives', '{}.png'.format(synapse['oid']))

    img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);
    synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, textXOffset, textYOffset)
    print(filename)

/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1823.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1825.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1841.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1867.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1868.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1884.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1885.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/1887.png
/Users/anish/Documents/Connectome/Synapt

/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3025.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3053.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3054.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3067.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3098.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3121.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3123.png
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/synaptograms/false_positives/3135.png
/Users/anish/Documents/Connectome/Synapt

In [84]:
for n in range(1, 2): 
    print(n) 

1


In [ ]:
603